In [1]:
import pandas as pd

df = pd.read_parquet('train-00000-of-00001-b64601da56687a05.parquet')

In [4]:
print(df.head())

                                               image  \
0  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...   
1  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...   
2  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...   
3  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...   
4  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...   

                                                text  
0  "a logo of coffee shop, take-away coffee cardb...  
1  "a logo of coffee shop, White round background...  
2  "a logo of coffee shop, image of a filled cup ...  
3  "a logo of cafe restaurant bar pizzeria with a...  
4  "a logo of cafe restaurant bar with a circle w...  


In [6]:
!pip install data model utils

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement model (from versions: none)
ERROR: No matching distribution found for model


In [5]:
import pandas as pd
import torch
from PIL import Image
from data import AttnDataset, resize_img
from model import RNN_ENCODER, G_NET
from utils import build_models, build_transform, gen_example

# Load your dataset
df = pd.read_parquet('your_dataset.parquet')

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained models
text_encoder, netG = build_models()

# Define image transformation
transform = build_transform()

# Define data loader
dataset = AttnDataset(df, transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)

# Generate images
for idx, (text, img_bytes) in enumerate(dataloader):
    # Preprocess text
    text = text[0]  # Extract text from batch
    text = text.lower()  # Convert text to lowercase (if necessary)
    
    # Preprocess image
    img_bytes = img_bytes[0]  # Extract image bytes from batch
    img = Image.open(io.BytesIO(img_bytes))
    img = resize_img(img, 64)  # Resize image to desired size (assuming 64x64)

    # Convert image to tensor and move to device
    img = transform(img).unsqueeze(0).to(device)

    # Tokenize text
    captions = [text.split()]
    cap_lens = [len(captions[0])]
    captions = torch.tensor(captions).to(device)
    cap_lens = torch.tensor(cap_lens).to(device)
    
    with torch.no_grad():
        _, _, _, _, fake_imgs, _, _ = gen_example(text_encoder, netG, captions, cap_lens)
    
    # Convert generated images from tensors to PIL images
    fake_imgs = fake_imgs[-1].cpu()
    fake_imgs = fake_imgs.view(-1, 3, 64, 64)  # Assuming image size is 64x64
    fake_imgs = fake_imgs.mul(0.5).add(0.5)
    fake_imgs = fake_imgs.mul(255).byte()
    
    # Save or display generated images
    for i in range(fake_imgs.size(0)):
        img = fake_imgs[i].permute(1, 2, 0)
        img = Image.fromarray(img.numpy())
        img.save(f"generated_image_{idx}_{i}.png")


ModuleNotFoundError: No module named 'data'